<a href="https://colab.research.google.com/github/baraabouzaiene/food-cv-app/blob/main/food_nutrition_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# COMPLETE FOOD NUTRITION PIPELINE


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import requests
import json
import numpy as np


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. LOAD YOUR TRAINED MODEL

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load model directly (not wrapped in a class)
vgg16_model = models.vgg16(pretrained=False)
vgg16_model.classifier[6] = torch.nn.Linear(4096, 101)

# Load your trained weights
model_path = '/content/drive/MyDrive/vgg16_food101.pth'
vgg16_model.load_state_dict(torch.load(model_path, map_location=device))
vgg16_model = vgg16_model.to(device)
vgg16_model.eval()

print(f"✅ Model loaded from: {model_path}")
print(f"✅ Device: {device}")


✅ Model loaded from: /content/drive/MyDrive/vgg16_food101.pth
✅ Device: cpu


# 2. PREPROCESSING & CLASSIFICATION


In [ ]:
print("\n📸 Setting up classification...")

# Same preprocessing as training
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Food-101 class names (alphabetical order)
FOOD_101_CLASSES = [
    'apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare',
    'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito',
    'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake',
    'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla',
    'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder',
    'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes',
    'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict',
    'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras',
    'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice',
    'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich',
    'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup',
    'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna',
    'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_soup',
    'mussels', 'nachos', 'omelette', 'onion_rings', 'oysters',
    'pad_thai', 'paella', 'pancakes', 'panna_cotta', 'peking_duck',
    'pho', 'pizza', 'pork_chop', 'poutine', 'prime_rib',
    'pulled_pork_sandwich', 'ramen', 'ravioli', 'red_velvet_cake', 'risotto',
    'samosa', 'sashimi', 'scallops', 'seaweed_salad', 'shrimp_and_grits',
    'spaghetti_bolognese', 'spaghetti_carbonara', 'spring_rolls', 'steak', 'strawberry_shortcake',
    'sushi', 'tacos', 'takoyaki', 'tiramisu', 'tuna_tartare', 'waffles'
]

def classify_food(image_path, model):
    """Classify food using trained VGG16"""
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        confidence, predicted_idx = torch.max(probabilities, 1)

    food_class = FOOD_101_CLASSES[predicted_idx.item()]
    confidence_score = confidence.item() * 100

    return food_class, confidence_score

print("✅ Classification ready!")



📸 Setting up classification...
✅ Classification ready!


# 3. NUTRITION API WITH FALLBACK


In [ ]:
print("\n🍎 Setting up nutrition API...")

USDA_API_KEY = "jJ4rbDZEmPdOkJmfs4eORBf5oZ4FbnULweay0ZK3"
USDA_BASE_URL = "https://api.nal.usda.gov/fdc/v1"

# Local fallback database
LOCAL_NUTRITION_DB = {
    'pizza': {'calories': 266, 'protein': 11, 'carbs': 33, 'fats': 10},
    'hamburger': {'calories': 295, 'protein': 17, 'carbs': 28, 'fats': 14},
    'donuts': {'calories': 452, 'protein': 5, 'carbs': 51, 'fats': 25},
    'apple_pie': {'calories': 237, 'protein': 2, 'carbs': 34, 'fats': 11},
    'sushi': {'calories': 143, 'protein': 6, 'carbs': 21, 'fats': 3.5},
    'chocolate_cake': {'calories': 371, 'protein': 5, 'carbs': 50, 'fats': 17},
    'french_fries': {'calories': 312, 'protein': 3.4, 'carbs': 41, 'fats': 15},
    'ice_cream': {'calories': 207, 'protein': 3.5, 'carbs': 24, 'fats': 11},
    'steak': {'calories': 271, 'protein': 25, 'carbs': 0, 'fats': 19},
    'caesar_salad': {'calories': 190, 'protein': 7, 'carbs': 10, 'fats': 14},
}

def get_nutrition_with_fallback(food_name):
    """Get nutrition from USDA API with local fallback"""
    # Try API first
    try:
        search_url = f"{USDA_BASE_URL}/foods/search"
        params = {'api_key': USDA_API_KEY, 'query': food_name, 'pageSize': 1}
        response = requests.get(search_url, params=params, timeout=5)
        data = response.json()

        if data.get('foods'):
            food_item = data['foods'][0]
            nutrients = {'calories': 0, 'protein': 0, 'carbs': 0, 'fats': 0}

            for nutrient in food_item.get('foodNutrients', []):
                name = nutrient.get('nutrientName', '').lower()
                value = nutrient.get('value', 0)

                if 'energy' in name:
                    nutrients['calories'] = round(value, 1)
                elif 'protein' in name and 'amino' not in name:
                    nutrients['protein'] = round(value, 1)
                elif 'carbohydrate' in name:
                    nutrients['carbs'] = round(value, 1)
                elif 'total lipid' in name:
                    nutrients['fats'] = round(value, 1)

            if nutrients['calories'] > 0:
                return nutrients, 'api'
    except:
        pass

    # Fallback to local database
    food_key = food_name.replace(' ', '_')
    nutrition = LOCAL_NUTRITION_DB.get(food_key, {'calories': 200, 'protein': 5, 'carbs': 25, 'fats': 8})
    return nutrition, 'local'

print(" Nutrition API ready!")




🍎 Setting up nutrition API...
 Nutrition API ready!


# 4. COMPLETE PIPELINE


In [ ]:
print("\n Complete pipeline ready!\n")

def analyze_food_complete(image_path):


    # Step 1: Classify food
    print("\n1️⃣  Classifying food...")
    food_name, confidence = classify_food(image_path, vgg16_model)
    print(f"   ✅ Detected: {food_name}")
    print(f"   📊 Confidence: {confidence:.1f}%")

    # Step 2: Get nutrition data
    print("\n2️⃣  Fetching nutrition data...")
    nutrition, source = get_nutrition_with_fallback(food_name)
    if source == 'api':
        print(f"   ✅ Retrieved from USDA API")
    else:
        print(f"   📚 Using local database")

    # Step 3: Estimate portion (default 100g for now)
    portion_grams = 100
    print(f"\n3️⃣  Portion size: {portion_grams}g (standard serving)")

    # Step 4: Compile results
    result = {
        'food_name': food_name,
        'confidence': round(confidence, 2),
        'portion_grams': portion_grams,
        'calories': nutrition['calories'],
        'protein': nutrition['protein'],
        'carbs': nutrition['carbs'],
        'fats': nutrition['fats'],
        'data_source': source
    }

    # Display results
    print("📊 NUTRITION RESULTS")
    print(f"Food:       {result['food_name']}")
    print(f"Confidence: {result['confidence']}%")
    print(f"Portion:    {result['portion_grams']}g")
    print(f"\n🥗 Nutritional Content:")
    print(f"   Calories:  {result['calories']} kcal")
    print(f"   Protein:   {result['protein']}g")
    print(f"   Carbs:     {result['carbs']}g")
    print(f"   Fats:      {result['fats']}g")
    print(f"\n📡 Data source: {result['data_source'].upper()}")
    print("="*60)

    return result





 Complete pipeline ready!

 ALL SYSTEMS READY!

1️⃣  Classifying food...
   ✅ Detected: apple_pie
   📊 Confidence: 49.2%

2️⃣  Fetching nutrition data...
   📚 Using local database

3️⃣  Portion size: 100g (standard serving)
📊 NUTRITION RESULTS
Food:       apple_pie
Confidence: 49.23%
Portion:    100g

🥗 Nutritional Content:
   Calories:  237 kcal
   Protein:   2g
   Carbs:     34g
   Fats:      11g

📡 Data source: LOCAL

1️⃣  Classifying food...
   ✅ Detected: donuts
   📊 Confidence: 60.9%

2️⃣  Fetching nutrition data...
   ✅ Retrieved from USDA API

3️⃣  Portion size: 100g (standard serving)
📊 NUTRITION RESULTS
Food:       donuts
Confidence: 60.88%
Portion:    100g

🥗 Nutritional Content:
   Calories:  404 kcal
   Protein:   3.5g
   Carbs:     61.4g
   Fats:      17.5g

📡 Data source: API


 ## testing


In [ ]:

result = analyze_food_complete('/content/cookies1.png')
result2 = analyze_food_complete('/content/donuts.png')

# Show result as JSON (for Django)
print("\n📱 JSON Output for API:")
print(json.dumps(result, indent=2))
print(json.dumps(result2, indent=2))


1️⃣  Classifying food...
   ✅ Detected: apple_pie
   📊 Confidence: 49.2%

2️⃣  Fetching nutrition data...
   📚 Using local database

3️⃣  Portion size: 100g (standard serving)
📊 NUTRITION RESULTS
Food:       apple_pie
Confidence: 49.23%
Portion:    100g

🥗 Nutritional Content:
   Calories:  237 kcal
   Protein:   2g
   Carbs:     34g
   Fats:      11g

📡 Data source: LOCAL

1️⃣  Classifying food...
   ✅ Detected: donuts
   📊 Confidence: 60.9%

2️⃣  Fetching nutrition data...
   ✅ Retrieved from USDA API

3️⃣  Portion size: 100g (standard serving)
📊 NUTRITION RESULTS
Food:       donuts
Confidence: 60.88%
Portion:    100g

🥗 Nutritional Content:
   Calories:  404 kcal
   Protein:   3.5g
   Carbs:     61.4g
   Fats:      17.5g

📡 Data source: API

📱 JSON Output for API:
{
  "food_name": "apple_pie",
  "confidence": 49.23,
  "portion_grams": 100,
  "calories": 237,
  "protein": 2,
  "carbs": 34,
  "fats": 11,
  "data_source": "local"
}
{
  "food_name": "donuts",
  "confidence": 60.88,
  "

# EXPORT FOR DJANGO BACKEND


In [ ]:
print(" EXPORTING FILES FOR DJANGO")

# 1. Export model to ONNX
print("\n   Converting model to ONNX...")
dummy_input = torch.randn(1, 3, 224, 224).to(device)

torch.onnx.export(
    vgg16_model,
    dummy_input,
    "vgg16_food101.onnx",
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)
print("    vgg16_food101.onnx created")

# 2. Export class names
print("\n  Exporting class names...")
with open('class_names.json', 'w') as f:
    json.dump(FOOD_101_CLASSES, f, indent=2)
print("    class_names.json created")

# 3. Export nutrition database
print("\n  Exporting nutrition database...")
with open('nutrition_database.json', 'w') as f:
    json.dump(LOCAL_NUTRITION_DB, f, indent=2)
print("    nutrition_database.json created")

# 4. Download all files
print("\n  Downloading files...")
from google.colab import files
files.download('vgg16_food101.onnx')
files.download('class_names.json')
files.download('nutrition_database.json')

print(" ALL FILES EXPORTED!")
print("\n Downloaded files:")
print("   • vgg16_food101.onnx (ML model)")
print("   • class_names.json (101 food classes)")
print("   • nutrition_database.json (fallback nutrition data)")
print("\n Ready for Django backend!")

 EXPORTING FILES FOR DJANGO

   Converting model to ONNX...


/tmp/ipython-input-2793508703.py:7: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


    vgg16_food101.onnx created

  Exporting class names...
    class_names.json created

  Exporting nutrition database...
    nutrition_database.json created



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 ALL FILES EXPORTED!

 Downloaded files:
   • vgg16_food101.onnx (ML model)
   • class_names.json (101 food classes)
   • nutrition_database.json (fallback nutrition data)

 Ready for Django backend!


In [ ]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 59.2 MB/s eta 0:00:00
